In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# (a)

# parameters
alpha = 0.33
beta = 0.95
delta = 0.025
gamma = 2
K = 250

# steady state capital
k_ss = ((1/(alpha*beta)) - (1-delta)/alpha)**(1/(alpha-1)) # formulation derived in text document
print('k_ss:', k_ss)

# capital grid
k_grid = (np.linspace(0.01*k_ss, 2*k_ss, K))
V_guess = (np.zeros(K))

In [ ]:
# (b)

def value_iteration(initial_guess, k_grid, alpha, beta, delta, gamma):
    
    V_next = (np.zeros(len(k_grid))) # to store updates to value function
    g = (np.zeros(len(k_grid))) # to store indices of the next capital on k_grid
    
    for initial_k_index in range(len(k_grid)):
        initial_k = k_grid[initial_k_index]
        max_value = -np.inf #initial benchmark to get updated

        for next_k_index in range(len(k_grid)):
            
            next_k = k_grid[next_k_index]

            consumption = max(0, initial_k**alpha + (1-delta)*initial_k - next_k)
            if consumption > 0:
                overall_value = ((consumption**(1-gamma))/(1-gamma)) + beta*initial_guess[next_k_index]
            else:
                overall_value = -np.inf

            if overall_value > max_value:
                max_value = overall_value
                V_next[initial_k_index] = max_value
                g[initial_k_index] = next_k_index
            else:
                pass

        
    return V_next, g

In [ ]:
# (c)

cutoff = 10**(-11)
V_next = value_iteration(V_guess, k_grid, 0.33, 0.95, 0.025, 2)[0]
while np.linalg.norm(V_next - V_guess) >= cutoff:
    V_guess = V_next
    V_next = value_iteration(V_guess, k_grid, 0.33, 0.95, 0.025, 2)[0]

In [ ]:
# (d)

value_function, policy_indices = value_iteration(V_guess, k_grid, 0.33, 0.95, 0.025, 2)
policy_indices = policy_indices.astype(int)
plt.figure(figsize=(10, 6))
plt.plot(k_grid, value_function, label='value function', color='blue')  

plt.xlabel('initial capital')
plt.ylabel('value function')
plt.title('value function')
plt.axhline(0, color='black', linewidth=0.5, ls='--')  
plt.axvline(0, color='black', linewidth=0.5, ls='--')  
plt.grid()
plt.legend()
plt.show()

In [ ]:
# (e)

policy_function = [k_grid[index] for index in policy_indices.astype(int)]

# 45-degree line for reference
line_x = np.linspace(0, max(k_grid), 100)
line_y = line_x  # y = x for 45-degree line

plt.figure(figsize=(10, 6))
plt.plot(k_grid, policy_function, label='policy function', color='blue')  
plt.plot(line_x, line_y, color='red', linestyle='--', label='45-Degree Line')

plt.xlabel('initial capital')
plt.ylabel('next_capital (policy function)')
plt.title('policy function')
plt.axhline(0, color='black', linewidth=0.5, ls='--')  
plt.axvline(0, color='black', linewidth=0.5, ls='--')  
plt.grid()
plt.legend()
plt.show()

In [ ]:
# (f)

# initializing the paths for each as zeros, which will get updated with the policy function
path_1 = np.zeros(100)
path_1[0] = k_grid[0]

path_K = np.zeros(100)
path_K[0] = k_grid[-1]

for t in range(len(path_1)-1):
    k_grid_index1 = int(list(k_grid).index(path_1[t]))
    path_1[t+1] = k_grid[policy_indices[k_grid_index1]]
    
    k_grid_indexK = int(list(k_grid).index(path_K[t]))
    path_K[t+1] = k_grid[policy_indices[k_grid_indexK]]

In [ ]:
# (g)

path_1_consumption = [path_1[t]**alpha + (1-delta)*path_1[t] - path_1[t+1] for t in range(len(path_1)-1)]
path_K_consumption = [path_K[t]**alpha + (1-delta)*path_K[t] - path_K[t+1] for t in range(len(path_K)-1)]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(path_1[:-1], path_1_consumption, label='path_1', color='blue')  
plt.plot(path_K[:-1], path_K_consumption, label='path_K', color='red') 


plt.xlabel('capital')
plt.ylabel('consumption')
plt.title('Phase diagram')
plt.axhline(0, color='black', linewidth=0.5, ls='--')  
plt.axvline(0, color='black', linewidth=0.5, ls='--')  
plt.grid()
plt.legend()
plt.show()